In [1]:
import panda_ep_dataset as ped
import panda_step_dataset as psd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
def as_gif(images, path="sample.gif"):
    images[0].save(path, save_all=True, append_images=images, duration=int(1000/30), loop=0)
    gif_bytes = open(path,"rb").read()
    return gif_bytes

# Step Dataset

In [ ]:
# Prints shapes each attribute and outputs a gif of the episode
panda_step_dataset = psd.PandaStepDataset(root_dir="./panda_dataset", ep_num=1, transform=psd.ToTensor())
dataloader = DataLoader(panda_step_dataset, batch_size=10, num_workers=5)
images = []
for i_batch, sample_batched in enumerate(dataloader):
    timestamp = sample_batched["timestamp"]
    joint_angles = sample_batched["joint_angles"]
    cam_in = sample_batched["intrinsic"]
    cam_ex = sample_batched["extrinsic"]
    image_batch = sample_batched["image"].squeeze().permute(0, 2, 3, 1).numpy()
    for image in image_batch:
        images.append(Image.fromarray(image))
    print(f"timestamp: {timestamp}")
    print(f"cam_in: {cam_in.shape}")
    print(f"cam_ex: {cam_ex.shape}")
    print(f"joint angles: {joint_angles.shape}")
    print(f"image shape: {image.shape}")
as_gif(images)


# Episode Dataset

In [4]:
# Episode dataset needs custom collate function for batching
def panda_custom_collate(batch):
    collated_batch = {}
    collated_intrinsic = torch.tensor([])
    collated_extrinsic = torch.tensor([])
    collated_timestamps = []
    collated_steps = []
    for item in batch:
        collated_intrinsic = torch.concatenate((collated_intrinsic, item["calibration_info"]["intrinsic"]))
        collated_extrinsic = torch.concatenate((collated_extrinsic, item["calibration_info"]["extrinsic"]))
        collated_timestamps.append(item["timestamps"])
        collated_steps.append(item["steps"])

    collated_batch["calibration_info"] = dict()
    collated_batch["calibration_info"]["intrinsic"] = collated_intrinsic
    collated_batch["calibration_info"]["extrinsic"] = collated_extrinsic
    collated_batch["timestamps"] = collated_timestamps
    collated_batch["steps"] = collated_steps
    
    return collated_batch

In [ ]:
panda_dataset = ped.PandaEpisodeDataset(root_dir="./panda_dataset", transform=ped.ToTensor())
dataloader = DataLoader(panda_dataset, batch_size=3, num_workers=5, collate_fn=panda_custom_collate, shuffle=False)
for i_batch, sample_batched in enumerate(dataloader):
    images = []
    timestamps = sample_batched["timestamps"]
    print(timestamps)
    for i, timestamp in enumerate(sample_batched["steps"]):
        print(len(sample_batched["steps"]))

In [ ]:
# Prints camera intrinsics and extrinsics and outputs a gif for each episode
panda_dataset = ped.PandaEpisodeDataset(root_dir="./panda_dataset", transform=ped.ToTensor())
dataloader = DataLoader(panda_dataset, batch_size=1, num_workers=0)
for i_batch, sample_batched in enumerate(dataloader):
    images = []
    timestamps = sample_batched["timestamps"]
    cam_in = sample_batched["calibration_info"]["intrinsic"][0]
    cam_ex = sample_batched["calibration_info"]["extrinsic"][0]
    print(f"Camera Intrinsics:\n{cam_in}")
    print(f"Camera Extrinsics:\n{cam_ex}")
    for i, timestamp in enumerate(sample_batched["steps"]):
        step = sample_batched["steps"][timestamp]
        image = step["img_data"].squeeze().permute(1,2,0).numpy()
        images.append(Image.fromarray(image))
    ep_num = str(i_batch+1).zfill(4)
    as_gif(images, path=f"./assets/gifs/panda_{ep_num}.gif")
